In [1]:
print("start");

In [2]:
!pip install opencc-python-reimplemented

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
from opencc import OpenCC

cc = OpenCC('s2twp')
text = '投票当天需携带投票通知单、国民身分证及印章，若没有收到投票通知书，可以向户籍所在地邻长查询投票所，印章则是可以用签名代替，至于身分证则是一定要携带。'

print(cc.convert(text))

投票當天需攜帶投票通知單、國民身分證及印章，若沒有收到投票通知書，可以向戶籍所在地鄰長查詢投票所，印章則是可以用簽名代替，至於身分證則是一定要攜帶。


In [4]:
import pandas as pd

FILE = 'data/conceptNet/chineseconceptnet.csv'
data = pd.read_csv(FILE, delimiter='\t')
data.columns = ['uri', 'relation', 'start', 'end', 'json']

In [5]:
# 删除不只含有中文节点的关系
data = data[data['start'].apply(lambda row: row.find('zh') > 0) & data['end'].apply(lambda row: row.find('zh') > 0)]
data.index = range(data.shape[0])
print(data)

                                             uri    relation       start  \
0             /a/[/r/Antonym/,/c/zh/上/,/c/zh/下/]  /r/Antonym     /c/zh/上   
1           /a/[/r/Antonym/,/c/zh/上/a/,/c/zh/下/]  /r/Antonym   /c/zh/上/a   
2           /a/[/r/Antonym/,/c/zh/上/r/,/c/zh/下/]  /r/Antonym   /c/zh/上/r   
3           /a/[/r/Antonym/,/c/zh/上/v/,/c/zh/下/]  /r/Antonym   /c/zh/上/v   
4         /a/[/r/Antonym/,/c/zh/上午/n/,/c/zh/下午/]  /r/Antonym  /c/zh/上午/n   
...                                          ...         ...         ...   
386561  /a/[/r/UsedFor/,/c/zh/鼻毛夾/,/c/zh/拔獅子鼻毛/]  /r/UsedFor   /c/zh/鼻毛夾   
386562       /a/[/r/UsedFor/,/c/zh/鼻涕/,/c/zh/哭/]  /r/UsedFor    /c/zh/鼻涕   
386563      /a/[/r/UsedFor/,/c/zh/鼻涕/,/c/zh/痛哭/]  /r/UsedFor    /c/zh/鼻涕   
386564     /a/[/r/UsedFor/,/c/zh/１_筆/,/c/zh/工作/]  /r/UsedFor   /c/zh/１_筆   
386565    /a/[/r/UsedFor/,/c/zh/ｍｐ_３/,/c/zh/聽歌/]  /r/UsedFor  /c/zh/ｍｐ_３   

                end                                               json  
0           /c

In [6]:
# 从json列中提取权重信息
import json

weights = data['json'].apply(lambda row: json.loads(row)['weight'])
data.pop('json')
data.insert(4, 'weights', weights)
print(data)

                                             uri    relation       start  \
0             /a/[/r/Antonym/,/c/zh/上/,/c/zh/下/]  /r/Antonym     /c/zh/上   
1           /a/[/r/Antonym/,/c/zh/上/a/,/c/zh/下/]  /r/Antonym   /c/zh/上/a   
2           /a/[/r/Antonym/,/c/zh/上/r/,/c/zh/下/]  /r/Antonym   /c/zh/上/r   
3           /a/[/r/Antonym/,/c/zh/上/v/,/c/zh/下/]  /r/Antonym   /c/zh/上/v   
4         /a/[/r/Antonym/,/c/zh/上午/n/,/c/zh/下午/]  /r/Antonym  /c/zh/上午/n   
...                                          ...         ...         ...   
386561  /a/[/r/UsedFor/,/c/zh/鼻毛夾/,/c/zh/拔獅子鼻毛/]  /r/UsedFor   /c/zh/鼻毛夾   
386562       /a/[/r/UsedFor/,/c/zh/鼻涕/,/c/zh/哭/]  /r/UsedFor    /c/zh/鼻涕   
386563      /a/[/r/UsedFor/,/c/zh/鼻涕/,/c/zh/痛哭/]  /r/UsedFor    /c/zh/鼻涕   
386564     /a/[/r/UsedFor/,/c/zh/１_筆/,/c/zh/工作/]  /r/UsedFor   /c/zh/１_筆   
386565    /a/[/r/UsedFor/,/c/zh/ｍｐ_３/,/c/zh/聽歌/]  /r/UsedFor  /c/zh/ｍｐ_３   

                end  weights  
0           /c/zh/下      1.0  
1           /c/zh/下      

In [7]:
# 查询起始节点
def search(words, n=20):
    result = data[data['start'].str.contains(cc.convert(words))]
    topK_result = result.sort_values("weights", ascending=False).head(n)
    return topK_result

In [10]:
template = {
    '/r/RelatedTo': '和{}相关',
    '/r/FormOf': '的形式为{}',
    '/r/IsA': '是{}',
    '/r/PartOf': '是{}的一部分',
    '/r/HasA': '具有{}',
    '/r/UsedFor': '用来{}',
    '/r/CapableOf': '可以{}',
    '/r/AtLocation': '在{}',
    '/r/Causes': '导致{}',
    '/r/HasSubevent': ',接下来,{}',
    '/r/HasFirstSubevent': '，紧接着，{}',
    '/r/HasLastSubevent': '的最后一步是{}',
    '/r/HasPrerequisite': '的前提为{}',
    '/r/HasProperty': '具有{}的属性',
    '/r/MotivatedByGoal': '受到{}的驱动',
    '/r/ObstructedBy': '受到{}的影响',
    '/r/Desires': '想要{}',
    '/r/CreatedBy': '被{}创造',
    '/r/Synonym': '和{}同义',
    '/r/Antonym': '和{}反义',
    '/r/DistinctFrom': '和{}相区别',
    '/r/DerivedFrom': '由{}导致',
    '/r/SymbolOf': '象征着{}',
    '/r/DefinedAs': '定义为{}',
    '/r/MannerOf': '',
    '/r/LocatedNear': '和{}相邻',
    '/r/HasContext': '的背景是{}',
    '/r/SimilarTo': '和{}相似',
    '/r/EtymologicallyRelatedTo': '',
    '/r/EtymologicallyDerivedFrom': '',
    '/r/CausesDesire': '',
    '/r/MadeOf': '由{}制成',
    '/r/ReceivesAction': '',
    '/r/ExternalURL': ''
}


def strip(str):
    return str.split('/')[3]

topK_result = search("快樂", 20)
for i in topK_result.index:
    i = topK_result.loc[i]
    if len(template[i['relation']]) > 0:
        fanti = strip(i['start']) + template[i['relation']].format(strip(i['end']))
        jianti = cc.convert(fanti)
        print(jianti)

快樂導致開心
快樂導致笑
快樂導致大笑
快樂,接下來,大笑
快樂導致唱歌
快樂是心情
快樂,接下來,唱歌
快樂是情緒
快樂導致跳舞
快樂是感覺
快樂導致幸福
快樂導致心情好
快樂受到開心的驅動
快樂,接下來,笑
快樂導致微笑
快樂導致滿足
快樂導致高興
快樂是幸福


In [11]:
print('end')

end
